In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain import hub
from langchain.vectorstores import Chroma
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
import pickle

Document loader

In [71]:
import glob

pdf_files = glob.glob("src/*.pdf")
pages = []

for pdf_file in pdf_files:
    pages.extend(PyPDFLoader(pdf_file).load_and_split())


Document(page_content="Imprimerie Officielle de la République Tunisienne51  Les personnes exerçant les métiers mentionnés à l’alinéa précédent du \nprésent article sont tenues de s'assur er pour leur responsabilité civile, celle \ndes personnes trava illant dans leur exploitation  et celle de toute personne \nayant la garde ou la  conduite des véhicule s dans le cadre de  leurs activités. \nArticle 111.- L'obligation d'assurance prévue à l'article 110 du \nprésent code s'applique à tout véhicule terrestre à moteur ainsi qu'à ses \nremorques à l'exception des véhicules appartenant à l'État et des \nvéhicules circulant sur les voies ferrées. \nArticle 112.- Les entreprises d'assurances agréées pour pratiquer \nl’assurance de la responsabilité civile du fait de l'usage des véhicules terrestres à moteur sont tenues de fournir cette assurance aux \npersonnes visées à l'article 110 du présent code. \nLe silence gardé par l'entreprise d'assurance pendant plus de dix \njours après  réception d

Splitter


In [72]:
doc_chunks = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    separators=["\nArticle", "\n\n", "\n", ".", "!", "?", ",", " ", ""],
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(pages)
len(chunks)

29084

Vectorstores

In [73]:
# import os
# os.environ["PINECONE_API_KEY"] = "9721efa4-ff98-4bc5-9b28-93919d1657a5"
embedder = HuggingFaceEmbeddings(
    model_name = "EIStakovskii/LaBSE-fr-de"
)

index_name = "pfa"

docsearch = PineconeVectorStore.from_documents(chunks, embedder, index_name=index_name)

# Retrieval
## check retriever

In [60]:

query="est-ce que un membre de la Chambre des députés peut aller en prison?"

#méthode 1 avec retriever
# retriever = docsearch.as_retriever(search_type="mmr")
# matched_docs = retriever.get_relevant_documents(query)
# for i, d in enumerate(matched_docs):
#     print(f"\n## Document {i}\n")
#     print(d.page_content)

#méthode 2 avec mmr
found_docs = docsearch.max_marginal_relevance_search(query, k=2, fetch_k=10)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n\n")

#méthode 3 avec similarity search
# sim_docs = docsearch.similarity_search_with_score(query, k=3)  
# for result in sim_docs:
#     print("\n")
#     print(result[1])
#     print(result[0].page_content)

1. par la Chambre des députés, et ce, jusqu' à ce qu'il soit possible de procéder aux 
élections. 
Le Président de la Répu blique est rééligible. 


2. Article 27 (Modifié par la loi constitu tionnelle n° 2002-51 du 1er juin 2002). 
Aucun membre de la Chambre des députés ou de la Chambre des conseillers ne 
peut, pendant la durée de son mandat, être poursuivi ou arrêté pour crime ou délit, 
tant que la Chambre concernée n'aura pas levé l'immunité qui le couvre. 
Toutefois, en cas de flagrant délit, il pe ut être procédé à son arrestation. La 
Chambre concernée en est informée sans délai. La détention est suspendue si la 




In [65]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



new_query= "contexte :  "+ format_docs(found_docs)+ "question : "+ query +" on parle toujours en tunisie et répondez en français"



In [66]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm.invoke(new_query)

'En Tunisie, les membres du Parlement (Chambre des Députés ou Chambre des Conseillers) sont protégés par l\'immunité parlementaire pendant la durée de leur mandat. Ce means that they cannot be arrested or prosecuted for crimes or misdemeanors during this time, unless there is evidence of a "flagrante delito" (a serious crime committed in flagrante delicto).\n\nIn such cases, the member can be arrested and detained, but only temporarily until the Chamber concerned lifts their immunity. The Chamber must be informed without delay of any such arrest. If the question is asked whether a member of the Chamber of Deputies can go to prison, the answer is always "no" in Tunisia, as long as they are enjoying immunities during their term in office.'